In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import ray
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from scipy import sparse
import logging

In [2]:
ray.init()

2021-05-26 12:57:24,579	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.31.36.235',
 'raylet_ip_address': '172.31.36.235',
 'redis_address': '172.31.36.235:6379',
 'object_store_address': '/tmp/ray/session_2021-05-26_12-57-23_694130_28713/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-26_12-57-23_694130_28713/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-26_12-57-23_694130_28713',
 'metrics_export_port': 53348,
 'node_id': '53244dc734d6b85951d051190d7e5566f353d2fe35cea4b3912cef20'}

In [3]:
DATA_DIR = "/mnt/efs/AmpC_data"

# Fingerprint molecules
The whole set of fingerprints won't fit in memory (even sparse) so we have to save them as chunks. This iterates over the SMILES codes, generating fingerprint_matrices and score arrays, saving them as chunks of 10,000,000

In [6]:
@ray.remote
def parse(line, i):
    if i % 1000 == 0:
        logging.basicConfig(level=logging.INFO)
        logging.info(i)
    
    
    if line == "":
        return
    
    zinc_id, smiles, score = line.split(",")
    if score == "no_score":
        return
    
    mol = Chem.MolFromSmiles(smiles)
    pars = { "radius": 2,
             "nBits": 8192,
             "invariants": [],
             "fromAtoms": [],
             "useChirality": False,
             "useBondTypes": True,
             "useFeatures": True,
    }
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, **pars)
    onbits = list(fp.GetOnBits())

    return onbits, float(score)

In [7]:
def get_fingerprints(fname, outFileName, fpSize=8192):
    with open(fname) as f:
        lines = f.read().split("\n")[1:]
        future_values = [parse.remote(line, i) for (i, line) in enumerate(lines)]
    
    values = [v for v in ray.get(future_values) if v]
    all_bits, scores = zip(*values)
    
    row_idx = []
    col_idx = []
    for i, bits in enumerate(all_bits):
        # these bits all have the same row:
        row_idx += [i] * len(bits)
        #and the column indices of those bits:
        col_idx += bits
    
    # generate a sparse matrix out of the row,col indices:
    unfolded_size = 8192
    fingerprint_matrix = sparse.coo_matrix((np.ones(len(row_idx)).astype(bool), (row_idx, col_idx)), 
                          shape=(max(row_idx)+1, unfolded_size))
    
    # convert to csr matrix, it is better:
    fingerprint_matrix =  sparse.csr_matrix(fingerprint_matrix)

    sparse.save_npz(outFileName+'.npz', fingerprint_matrix)
    np.save(outFileName+'.npy', np.array(scores))

# Count number of valid molecules:

In [8]:
fname = f"{DATA_DIR}/data/AmpC_screen_table.csv"

In [9]:
def count_valid_molecules(fname):
    with open(fname) as fileobj:
        fileobj.readline()
        count = 0
        for line in fileobj:
            words = line[:-1].split(',')
            if len(words[2]) < 1:
                continue
            if words[2] == 'no_score':
                break
            count += 1
        fileobj.close()

In [10]:
# count_valid_molecules(fname=fname)

In [ ]:
# for i in range( np.ceil(count / chunksize).astype(int) ):
get_fingerprints(fname, f"{DATA_DIR}/processed_data/AmpC_all")

(pid=28856) INFO:root:0
(pid=28854) INFO:root:1000
(pid=28853) INFO:root:2000
(pid=28852) INFO:root:3000
(pid=28849) INFO:root:4000
(pid=28866) INFO:root:5000
(pid=28850) INFO:root:6000
(pid=28857) INFO:root:7000
(pid=28855) INFO:root:8000
(pid=28855) INFO:root:9000
(pid=28862) INFO:root:10000
(pid=28855) INFO:root:11000
(pid=28852) INFO:root:12000
(pid=28853) INFO:root:13000
(pid=28850) INFO:root:14000
(pid=28866) INFO:root:15000
(pid=28852) INFO:root:16000
(pid=28855) INFO:root:17000
(pid=28852) INFO:root:18000
(pid=28854) INFO:root:19000
(pid=28863) INFO:root:20000
(pid=28861) INFO:root:21000
(pid=28856) INFO:root:22000
(pid=28856) INFO:root:23000
(pid=28860) INFO:root:24000
(pid=28852) INFO:root:25000
(pid=28861) INFO:root:26000
(pid=28853) INFO:root:27000
(pid=28863) INFO:root:28000
(pid=28853) INFO:root:29000
(pid=28857) INFO:root:30000
(pid=28850) INFO:root:31000
(pid=28859) INFO:root:32000
(pid=28858) INFO:root:33000
(pid=28862) INFO:root:34000
(pid=28853) INFO:root:35000
(pid=

(pid=28852) INFO:root:199000
(pid=28862) INFO:root:200000
(pid=28857) INFO:root:201000
(pid=28853) INFO:root:202000
(pid=28852) INFO:root:203000
(pid=28863) INFO:root:204000
(pid=28861) INFO:root:205000
(pid=28853) INFO:root:206000
(pid=28862) INFO:root:207000
(pid=28851) INFO:root:208000
(pid=28850) INFO:root:209000
(pid=28853) INFO:root:210000
(pid=28849) INFO:root:211000
(pid=28852) INFO:root:212000
(pid=28855) INFO:root:213000
(pid=28850) INFO:root:214000
(pid=28855) INFO:root:215000
(pid=28866) INFO:root:216000
(pid=28866) INFO:root:217000
(pid=28863) INFO:root:218000
(pid=28863) INFO:root:219000
(pid=28854) INFO:root:220000
(pid=28863) INFO:root:221000
(pid=28852) INFO:root:222000
(pid=28863) INFO:root:223000
(pid=28857) INFO:root:224000
(pid=28849) INFO:root:225000
(pid=28858) INFO:root:226000
(pid=28855) INFO:root:227000
(pid=28862) INFO:root:228000
(pid=28854) INFO:root:229000
(pid=28863) INFO:root:230000
(pid=28862) INFO:root:231000
(pid=28861) INFO:root:232000
(pid=28854) IN

(pid=28866) INFO:root:395000
(pid=28853) INFO:root:396000
(pid=28861) INFO:root:397000
(pid=28855) INFO:root:398000
(pid=28860) INFO:root:399000
(pid=28863) INFO:root:400000
(pid=28860) INFO:root:401000
(pid=28852) INFO:root:402000
(pid=28849) INFO:root:403000
(pid=28851) INFO:root:404000
(pid=28859) INFO:root:405000
(pid=28860) INFO:root:406000
(pid=28860) INFO:root:407000
(pid=28857) INFO:root:408000
(pid=28860) INFO:root:409000
(pid=28855) INFO:root:410000
(pid=28852) INFO:root:411000
(pid=28855) INFO:root:412000
(pid=28861) INFO:root:413000
(pid=28861) INFO:root:414000
(pid=28857) INFO:root:415000
(pid=28851) INFO:root:416000
(pid=28858) INFO:root:417000
(pid=28855) INFO:root:418000
(pid=28851) INFO:root:419000
(pid=28849) INFO:root:420000
(pid=28851) INFO:root:421000
(pid=28854) INFO:root:422000
(pid=28853) INFO:root:423000
(pid=28859) INFO:root:424000
(pid=28856) INFO:root:425000
(pid=28855) INFO:root:426000
(pid=28857) INFO:root:427000
(pid=28854) INFO:root:428000
(pid=28858) IN

(pid=28866) INFO:root:593000
(pid=28859) INFO:root:594000
(pid=28857) INFO:root:595000
(pid=28863) INFO:root:596000
(pid=28860) INFO:root:597000
(pid=28851) INFO:root:598000
(pid=28861) INFO:root:599000
(pid=28853) INFO:root:600000
(pid=28862) INFO:root:601000
(pid=28856) INFO:root:602000
(pid=28858) INFO:root:603000
(pid=28849) INFO:root:604000
(pid=28855) INFO:root:605000
(pid=28854) INFO:root:606000
(pid=28866) INFO:root:607000
(pid=28854) INFO:root:608000
(pid=28858) INFO:root:609000
(pid=28849) INFO:root:610000
(pid=28855) INFO:root:611000
(pid=28855) INFO:root:612000
(pid=28855) INFO:root:613000
(pid=28860) INFO:root:614000
(pid=28862) INFO:root:615000
(pid=28858) INFO:root:616000
(pid=28866) INFO:root:617000
(pid=28860) INFO:root:618000
(pid=28862) INFO:root:619000
(pid=28853) INFO:root:620000
(pid=28861) INFO:root:621000
(pid=28861) INFO:root:622000
(pid=28854) INFO:root:623000
(pid=28866) INFO:root:624000
(pid=28860) INFO:root:625000
(pid=28856) INFO:root:626000
(pid=28850) IN

(pid=28859) INFO:root:790000
(pid=28857) INFO:root:791000
(pid=28858) INFO:root:792000
(pid=28850) INFO:root:793000
(pid=28854) INFO:root:794000
(pid=28854) INFO:root:795000
(pid=28863) INFO:root:796000
(pid=28860) INFO:root:797000
(pid=28860) INFO:root:798000
(pid=28850) INFO:root:799000
(pid=28857) INFO:root:800000
(pid=28857) INFO:root:801000
(pid=28854) INFO:root:802000
(pid=28859) INFO:root:803000
(pid=28857) INFO:root:804000
(pid=28853) INFO:root:805000
(pid=28857) INFO:root:806000
(pid=28852) INFO:root:807000
(pid=28853) INFO:root:808000
(pid=28849) INFO:root:809000
(pid=28849) INFO:root:810000
(pid=28853) INFO:root:811000
(pid=28863) INFO:root:812000
(pid=28866) INFO:root:813000
(pid=28856) INFO:root:814000
(pid=28855) INFO:root:815000
(pid=28861) INFO:root:816000
(pid=28861) INFO:root:817000
(pid=28856) INFO:root:818000
(pid=28861) INFO:root:819000
(pid=28863) INFO:root:820000
(pid=28861) INFO:root:821000
(pid=28853) INFO:root:822000
(pid=28862) INFO:root:823000
(pid=28855) IN

...

(pid=28852) INFO:root:1178000
(pid=28859) INFO:root:1179000
(pid=28852) INFO:root:1180000
(pid=28860) INFO:root:1181000
(pid=28853) INFO:root:1182000
(pid=28866) INFO:root:1183000
(pid=28860) INFO:root:1184000
(pid=28853) INFO:root:1185000
(pid=28852) INFO:root:1186000
(pid=28849) INFO:root:1187000
(pid=28854) INFO:root:1188000
(pid=28850) INFO:root:1189000
(pid=28849) INFO:root:1190000
(pid=28858) INFO:root:1191000
(pid=28859) INFO:root:1192000
(pid=28857) INFO:root:1193000
(pid=28862) INFO:root:1194000
(pid=28860) INFO:root:1195000
(pid=28853) INFO:root:1196000
(pid=28856) INFO:root:1197000
(pid=28862) INFO:root:1198000
(pid=28850) INFO:root:1199000
(pid=28860) INFO:root:1200000
(pid=28856) INFO:root:1201000
(pid=28863) INFO:root:1202000
(pid=28858) INFO:root:1203000
(pid=28850) INFO:root:1204000
(pid=28863) INFO:root:1205000
(pid=28853) INFO:root:1206000
(pid=28859) INFO:root:1207000
(pid=28863) INFO:root:1208000
(pid=28849) INFO:root:1209000
(pid=28855) INFO:root:1210000
(pid=28853

(pid=28859) INFO:root:1370000
(pid=28856) INFO:root:1371000
(pid=28860) INFO:root:1372000
(pid=28866) INFO:root:1373000
(pid=28866) INFO:root:1374000
(pid=28859) INFO:root:1375000
(pid=28866) INFO:root:1376000
(pid=28866) INFO:root:1377000
(pid=28859) INFO:root:1378000
(pid=28860) INFO:root:1379000
(pid=28866) INFO:root:1380000
(pid=28855) INFO:root:1381000
(pid=28863) INFO:root:1382000
(pid=28860) INFO:root:1383000
(pid=28861) INFO:root:1384000
(pid=28856) INFO:root:1385000
(pid=28857) INFO:root:1386000
(pid=28859) INFO:root:1387000
(pid=28856) INFO:root:1388000
(pid=28851) INFO:root:1389000
(pid=28858) INFO:root:1390000
(pid=28855) INFO:root:1391000
(pid=28856) INFO:root:1392000
(pid=28858) INFO:root:1393000
(pid=28850) INFO:root:1394000
(pid=28851) INFO:root:1395000
(pid=28850) INFO:root:1396000
(pid=28861) INFO:root:1397000
(pid=28861) INFO:root:1398000
(pid=28854) INFO:root:1399000
(pid=28858) INFO:root:1400000
(pid=28856) INFO:root:1401000
(pid=28852) INFO:root:1402000
(pid=28858

(pid=28863) INFO:root:1563000
(pid=28858) INFO:root:1564000
(pid=28857) INFO:root:1565000
(pid=28862) INFO:root:1566000
(pid=28866) INFO:root:1567000
(pid=28853) INFO:root:1568000
(pid=28860) INFO:root:1569000
(pid=28858) INFO:root:1570000
(pid=28849) INFO:root:1571000
(pid=28849) INFO:root:1572000
(pid=28851) INFO:root:1573000
(pid=28853) INFO:root:1574000
(pid=28849) INFO:root:1575000
(pid=28850) INFO:root:1576000
(pid=28854) INFO:root:1577000
(pid=28857) INFO:root:1578000
(pid=28862) INFO:root:1579000
(pid=28862) INFO:root:1580000
(pid=28859) INFO:root:1581000
(pid=28851) INFO:root:1582000
(pid=28866) INFO:root:1583000
(pid=28849) INFO:root:1584000
(pid=28854) INFO:root:1585000
(pid=28858) INFO:root:1586000
(pid=28849) INFO:root:1587000
(pid=28850) INFO:root:1588000
(pid=28863) INFO:root:1589000
(pid=28866) INFO:root:1590000
(pid=28853) INFO:root:1591000
(pid=28862) INFO:root:1592000
(pid=28849) INFO:root:1593000
(pid=28852) INFO:root:1594000
(pid=28851) INFO:root:1595000
(pid=28853

(pid=28856) INFO:root:1754000
(pid=28854) INFO:root:1755000
(pid=28861) INFO:root:1756000
(pid=28859) INFO:root:1757000
(pid=28853) INFO:root:1758000
(pid=28851) INFO:root:1759000
(pid=28859) INFO:root:1760000
(pid=28854) INFO:root:1761000
(pid=28853) INFO:root:1762000
(pid=28856) INFO:root:1763000
(pid=28859) INFO:root:1764000
(pid=28854) INFO:root:1765000
(pid=28857) INFO:root:1766000
(pid=28866) INFO:root:1767000
(pid=28866) INFO:root:1768000
(pid=28849) INFO:root:1769000
(pid=28855) INFO:root:1770000
(pid=28858) INFO:root:1771000
(pid=28853) INFO:root:1772000
(pid=28855) INFO:root:1773000
(pid=28851) INFO:root:1774000
(pid=28852) INFO:root:1775000
(pid=28849) INFO:root:1776000
(pid=28866) INFO:root:1777000
(pid=28851) INFO:root:1778000
(pid=28853) INFO:root:1779000
(pid=28861) INFO:root:1780000
(pid=28852) INFO:root:1781000
(pid=28857) INFO:root:1782000
(pid=28850) INFO:root:1783000
(pid=28854) INFO:root:1784000
(pid=28851) INFO:root:1785000
(pid=28851) INFO:root:1786000
(pid=28849

(pid=28855) INFO:root:1946000
(pid=28853) INFO:root:1947000
(pid=28855) INFO:root:1948000
(pid=28866) INFO:root:1949000
(pid=28862) INFO:root:1950000
(pid=28863) INFO:root:1951000
(pid=28855) INFO:root:1952000
(pid=28853) INFO:root:1953000
(pid=28860) INFO:root:1954000
(pid=28855) INFO:root:1955000
(pid=28856) INFO:root:1956000
(pid=28858) INFO:root:1957000
(pid=28861) INFO:root:1958000
(pid=28860) INFO:root:1959000
(pid=28866) INFO:root:1960000
(pid=28849) INFO:root:1961000
(pid=28849) INFO:root:1962000
(pid=28851) INFO:root:1963000
(pid=28861) INFO:root:1964000
(pid=28858) INFO:root:1965000
(pid=28854) INFO:root:1966000
(pid=28863) INFO:root:1967000
(pid=28866) INFO:root:1968000
(pid=28851) INFO:root:1969000
(pid=28851) INFO:root:1970000
(pid=28862) INFO:root:1971000


In [16]:
!ls /mnt/efs/AmpC_data/data

AmpC_screen_table.csv  AmpC_screen_table.csv.zip
